In [1]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [10]:
class Joke(BaseModel):
    """Broma para contar al usuario.."""

    setup: str = Field(description="pregunta para montar una broma")
    punchline: str = Field(description="respuesta para resolver la broma")

In [11]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192").bind_tools([Joke])
llm

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017FCBEFD7E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017FCBEFDBA0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Joke', 'description': 'Broma para contar al usuario..', 'parameters': {'type': 'object', 'properties': {'setup': {'description': 'pregunta para montar una broma', 'type': 'string'}, 'punchline': {'description': 'respuesta para resolver la broma', 'type': 'string'}}, 'required': ['setup', 'punchline']}}}]})

In [12]:
llm.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Joke',
   'description': 'Broma para contar al usuario..',
   'parameters': {'type': 'object',
    'properties': {'setup': {'description': 'pregunta para montar una broma',
      'type': 'string'},
     'punchline': {'description': 'respuesta para resolver la broma',
      'type': 'string'}},
    'required': ['setup', 'punchline']}}}]

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "Eres un asistente útil"), ("user", "{input}")]
)

In [14]:
from typing import List
from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

In [3]:
parser = JsonOutputKeyToolsParser(key_name="Joke")

In [15]:
chain = prompt | llm | parser
chain

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Eres un asistente útil')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000017FCBEFD7E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000017FCBEFDBA0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Joke', 'description': 'Broma para contar al usuario..', 'parameters': {'type': 'object', 'properties': {'setup': {'description': 'pregunta para montar una broma', 'type': 'string'}, 'punchline': {'description': 'respuesta para resolver la broma', 'type': 'string'}}, 'required': ['setup', 'punchline']}}}]})
| JsonOutputKeyToolsParser(key_name='Joke')

In [16]:
resultado = chain.invoke({"input": "cuéntame un chiste"})
resultado

[{'setup': '¿Por qué la computadora fue al médico?',
  'punchline': '¡Porque tenía un virus!'}]

In [17]:
parser = JsonOutputKeyToolsParser(key_name="Joke", first_tool_only=True)
chain = prompt | llm | parser
resultado = chain.invoke({"input": "cuéntame un chiste"})
resultado

{'setup': '¿Por qué la computadora fue al médico?',
 'punchline': '¡Porque tenía un virus!'}